In [5]:
# import all necessary packages
import pandas as pd
import numpy as np
import json
import datetime
import warnings
from pandas.core.common import SettingWithCopyWarning

#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
#import seaborn as sns
#import matplotlib as mlp
import matplotlib.pyplot as plt
#from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.svm import SVR
import math
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from scipy.stats import loguniform

import predictive as pred

In [2]:
# import datasets
df_2h_comar = pd.read_csv("data/datasets/df_2h_comar.csv")
df_2h_hexlow = pd.read_csv("data/datasets/df_2h_hexlow.csv")
df_2h_hexmed = pd.read_csv("data/datasets/df_2h_hexmed.csv")
df_2h_hexhig = pd.read_csv("data/datasets/df_2h_hexhig.csv")

In [3]:
df_2h_comar.drop(columns = ['date_start'],inplace = True)
df_2h_hexlow.drop(columns = ['date_start'],inplace = True)
df_2h_hexmed.drop(columns = ['date_start'],inplace = True)
df_2h_hexhig.drop(columns = ['date_start'],inplace = True)

In [6]:
pred.dayandseas_to_int(df_2h_comar)

,start_stamp,Pickup_Community_Area,numOfTaxis_area,start_time_hourly,dayOfWeek,isHoliday,isRushhour,start_time_day,start_time_week,start_time_month,pressure,humidity,wind_direction,wind_speed,description,season,temperature_celsius
0,0,1.0,12,0,1,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,1,-0.19
1,0,2.0,4,0,1,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,1,-0.19
2,0,3.0,98,0,1,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,1,-0.19
3,0,4.0,38,0,1,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,1,-0.19
4,0,5.0,53,0,1,True,False,1,1,1,1024.0,64.0,200.0,4.0,overcast clouds,1,-0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136439,22,59.0,1,23,1,False,False,31,1,12,1023.0,89.0,217.0,1.0,mist,1,-11.00
136440,22,60.0,4,22,1,False,False,31,1,12,1026.0,89.0,241.0,1.0,snow,1,-11.24
136441,22,61.0,1,23,1,False,False,31,1,12,1023.0,89.0,217.0,1.0,mist,1,-11.00
136442,22,76.0,11,22,1,False,False,31,1,12,1026.0,89.0,241.0,1.0,snow,1,-11.24
